In [1]:
from google.colab import drive
import pandas as pd
drive.mount('/content/drive')
df_train = pd.read_csv("/content/drive/MyDrive/lable/001.csv",header=None)
df_train.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,0,1,2
0,親愛的朋友，感謝您蒞臨疾病管制署「COVID-19 公費疫苗預約平台」（以下簡稱本網站），您...,[],0
1,為了幫助您瞭解本署如何蒐集、處理、利用及保護您所提供的個人資訊，請您務必詳細閱讀下列資訊：,[],0
2,"Dear friends, thank you for visiting the “Gove...",[],0
3,本隱私權公告（以下簡稱本公告），適用於您在本網站辦理疫苗意願登記及預約接種服務（以下簡稱本服...,[],0
4,This privacy notice (hereinafter referred to a...,[],0


In [11]:
from glob import glob

files=glob('/content/drive/MyDrive/lable/*.csv')
types_dict={"text":str,"what":str,"label":int}
print(len(files))
print(files)
df=pd.concat((pd.read_csv(file,header=None,dtype=types_dict,names=['text','what','label'])for file in files),ignore_index=True)
print(df)

41
['/content/drive/MyDrive/lable/034.csv', '/content/drive/MyDrive/lable/035.csv', '/content/drive/MyDrive/lable/001.csv', '/content/drive/MyDrive/lable/040.csv', '/content/drive/MyDrive/lable/038.csv', '/content/drive/MyDrive/lable/028.csv', '/content/drive/MyDrive/lable/029.csv', '/content/drive/MyDrive/lable/032.csv', '/content/drive/MyDrive/lable/031.csv', '/content/drive/MyDrive/lable/041.csv', '/content/drive/MyDrive/lable/016.csv', '/content/drive/MyDrive/lable/012.csv', '/content/drive/MyDrive/lable/005.csv', '/content/drive/MyDrive/lable/013.csv', '/content/drive/MyDrive/lable/014.csv', '/content/drive/MyDrive/lable/002.csv', '/content/drive/MyDrive/lable/010.csv', '/content/drive/MyDrive/lable/039.csv', '/content/drive/MyDrive/lable/025.csv', '/content/drive/MyDrive/lable/022.csv', '/content/drive/MyDrive/lable/009.csv', '/content/drive/MyDrive/lable/019.csv', '/content/drive/MyDrive/lable/003.csv', '/content/drive/MyDrive/lable/024.csv', '/content/drive/MyDrive/lable/020.cs

In [12]:
df = df.drop(['what'],axis=1)

In [13]:
empty_title = (df["text"].isnull())
df = df[~empty_title]
df

,text,label
0,WOMEN,0
1,MEN,0
2,KIDS,0
3,BABY,0
4,SPORTS,0
...,...,...
5707,就個人資料之處理提出詢問及申訴聯絡表單,0
5708,10.b.iii.本公司處理個人資料的方法,0
5709,本公司遵守相關法規的方法及實施的安全措施安全及隱私,0
5710,安全使用本服務的秘訣安全中心,0


In [14]:
short_title=df[df['text'].map(len) <= 5]
df=df.drop(list(short_title.index))
df

,text,label
4,SPORTS,0
10,米格國際股份有限公司相關網站，包括但不限於：lativ網站（www.lativ.com.tw...,0
11,本公司十分重視您的隱私權保護，將依個人資料保護法及本隱私權政策蒐集、處理及利用您的個人資料，...,0
12,若您不同意本隱私權政策之全部或部份者，請您停止使用本網站服務。,0
14,本隱私權政策適用之範圍請您在於使用本網站服務前，確認您已審閱並同意本隱私權政策所列全部條款，...,0
...,...,...
5707,就個人資料之處理提出詢問及申訴聯絡表單,0
5708,10.b.iii.本公司處理個人資料的方法,0
5709,本公司遵守相關法規的方法及實施的安全措施安全及隱私,0
5710,安全使用本服務的秘訣安全中心,0


In [23]:
long_title=df[df['text'].map(len) >= 400]
df2=df.drop(list(long_title.index))
df2.label.value_counts()

0    4652
1      85
Name: label, dtype: int64

In [24]:
df.label.value_counts()

0    4661
1      87
Name: label, dtype: int64

In [25]:
!pip install transformers

     |████████████████████████████████| 2.6 MB 9.3 MB/s 
     |████████████████████████████████| 895 kB 67.5 MB/s 
     |████████████████████████████████| 636 kB 77.9 MB/s 
     |████████████████████████████████| 3.3 MB 62.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [26]:
import torch
from transformers import BertTokenizer
from IPython.display import clear_output

PRETRAINED_MODEL_NAME = "bert-base-chinese"
tokenizer = BertTokenizer.from_pretrained(PRETRAINED_MODEL_NAME)
clear_output()
print("PyTorch 版本：", torch.__version__)

PyTorch 版本： 1.9.0+cu102


In [27]:
train_text=list(df["text"])
train_label=list(df["label"])

In [28]:
train_text[110]

'您可隨時聯絡宏碁，拒絕我們使用您的個人資訊。'

In [29]:
text_tokenized=tokenizer(train_text[0], padding=True, truncation=True, max_length=512)
ids=text_tokenized["input_ids"]
type_ids=text_tokenized["token_type_ids"]
mask=text_tokenized["attention_mask"]
combined_text=tokenizer.convert_ids_to_tokens(ids)
print(f"ids:{torch.tensor(ids)}  句子:{combined_text}")

ids:tensor([101, 100, 102])  句子:['[CLS]', '[UNK]', '[SEP]']


In [30]:
from torch.utils.data import Dataset
class ClauseDataset(Dataset):
  def __init__(self,text,tokenizer,label=None):
    self.text=text
    self.label=label
    self.tokenizer=tokenizer
    self.len=len(self.text)
  def __getitem__(self,idx):
    #print("idx:",idx)
    tokens=tokenizer(self.text[idx],padding=True,truncation=True,max_length=512)
    ids=tokens["input_ids"]
    segments=tokens["token_type_ids"]
    masks =tokens["attention_mask"]
    tokens_tensor=torch.tensor(ids)
    segments_tensor=torch.tensor(segments)
    masks_tensors=torch.tensor(masks)
    label_tensor=torch.tensor(self.label[idx])

    return(tokens_tensor, segments_tensor, masks_tensors, label_tensor)
  def __len__(self):
    return self.len

trainset=ClauseDataset(train_text,label=train_label,tokenizer=tokenizer)

In [31]:
sample_idx = 110

# 將原始文本拿出做比較
text_a=trainset.text[sample_idx]
label=train_label[sample_idx]

# 利用剛剛建立的 Dataset 取出轉換後的 id tensors
tokens_tensor, segments_tensor, masks_tensors, label_tensor = trainset[sample_idx]

# 將 tokens_tensor 還原成文本
tokens = tokenizer.convert_ids_to_tokens(tokens_tensor.tolist())
combined_text = "".join(tokens)

# 渲染前後差異，毫無反應就是個 print。可以直接看輸出結果
print(f"""[原始文本]
句子 ：{text_a}
分類  ：{label}

--------------------

[Dataset 回傳的 tensors]
tokens_tensor  ：{tokens_tensor}

segments_tensor：{segments_tensor}

masks_tensors: {masks_tensors}

label_tensor   ：{label_tensor}

--------------------

[還原 tokens_tensors]
{combined_text}
""")

[原始文本]
句子 ：您可隨時聯絡宏碁，拒絕我們使用您的個人資訊。
分類  ：0

--------------------

[Dataset 回傳的 tensors]
tokens_tensor  ：tensor([ 101, 2644, 1377, 7401, 3229, 5474, 5181, 2131, 4805, 8024, 2867, 5179,
        2769,  947,  886, 4500, 2644, 4638,  943,  782, 6536, 6244,  511,  102])

segments_tensor：tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

masks_tensors: tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

label_tensor   ：0

--------------------

[還原 tokens_tensors]
[CLS]您可隨時聯絡宏碁，拒絕我們使用您的個人資訊。[SEP]



In [32]:
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

def create_mini_batch(samples):
  tokens_tensors = [s[0] for s in samples]
  segments_tensors = [s[1] for s in samples]
  masks_tensors =[s[2] for s in samples]
  if samples[0][3] is not None:
    label_ids = torch.stack([s[3] for s in samples])
  else:
    label_ids = None
  tokens_tensors = pad_sequence(tokens_tensors,batch_first=True)
  segments_tensors = pad_sequence(segments_tensors,batch_first=True)
  masks_tensors = pad_sequence(masks_tensors,batch_first=True)

  return tokens_tensors, segments_tensors, masks_tensors, label_ids

BATCH_SIZE = 8
trainloader = DataLoader(trainset, batch_size=BATCH_SIZE, collate_fn=create_mini_batch)

In [33]:
data = next(iter(trainloader))

tokens_tensors, segments_tensors, \
    masks_tensors, label_ids = data

print(f"""
tokens_tensors.shape   = {tokens_tensors.shape} 
{tokens_tensors}
------------------------
segments_tensors.shape = {segments_tensors.shape}
{segments_tensors}
------------------------
masks_tensors.shape    = {masks_tensors.shape}
{masks_tensors}
------------------------
label_ids.shape        = {label_ids.shape}
{label_ids}
""")


tokens_tensors.shape   = torch.Size([8, 87]) 
tensor([[ 101,  100,  102,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0],
        [ 101, 5101, 3419, 1751, 7396, 5500,  819, 3300, 7361, 1062, 1385, 4685,
         7302, 5206, 4991, 8024, 1259, 2886,  852,  679, 7361, 3176, 8038, 8515,
         9007, 8225, 5206, 4991, 8020, 8173,  119, 8515, 9007, 8225,  119, 8134,
          119, 8351, 8021,  510,  100, 2398, 1378, 1350, 2527, 5265, 2768, 4989,
          722, 5206, 4991, 5023, 8

In [34]:
from transformers import BertForSequenceClassification
PRETRAINED_MODEL_NAME = "bert-base-chinese"
NUM_LABELS = 2
model = BertForSequenceClassification.from_pretrained(PRETRAINED_MODEL_NAME, num_labels=NUM_LABELS)
clear_output()

print("""
name            module
----------------------""")
for name, module in model.named_children():
    if name == "bert":
        for n, _ in module.named_children():
            print(f"{name}:{n}")
    else:
        print("{:15} {}".format(name, module))


name            module
----------------------
bert:embeddings
bert:encoder
bert:pooler
dropout         Dropout(p=0.1, inplace=False)
classifier      Linear(in_features=768, out_features=2, bias=True)


In [35]:
def get_predictions(model, dataloader, compute_acc=False):
    predictions = None
    correct = 0
    total = 0
      
    with torch.no_grad():
        # 遍巡整個資料集
        for data in dataloader:
            # 將所有 tensors 移到 GPU 上
            if next(model.parameters()).is_cuda:
                data = [t.to("cuda:0") for t in data if t is not None]
            
            
            # 別忘記前 3 個 tensors 分別為 tokens, segments 以及 masks
            # 且強烈建議在將這些 tensors 丟入 `model` 時指定對應的參數名稱
            tokens_tensors, segments_tensors, masks_tensors = data[:3]
            outputs = model(input_ids=tokens_tensors, 
                            token_type_ids=segments_tensors, 
                            attention_mask=masks_tensors)
            
            logits = outputs[0]
            _, pred = torch.max(logits.data, 1)
            
            # 用來計算訓練集的分類準確率
            if compute_acc:
                labels = data[3]
                total += labels.size(0)
                correct += (pred == labels).sum().item()
                
            # 將當前 batch 記錄下來
            if predictions is None:
                predictions = pred
            else:
                predictions = torch.cat((predictions, pred))
    
    if compute_acc:
        acc = correct / total
        return predictions, acc
    return predictions
    
# 讓模型跑在 GPU 上並取得訓練集的分類準確率
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("device:", device)
model = model.to(device)
#_, acc = get_predictions(model, trainloader, compute_acc=True)
#print("classification acc:", acc)

device: cuda:0


In [36]:
%%time

# 訓練模式
model.train()

# 使用 Adam Optim 更新整個分類模型的參數
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)


EPOCHS = 6  # 幸運數字
for epoch in range(EPOCHS):
    
    running_loss = 0.0
    for data in trainloader:
        
        tokens_tensors, segments_tensors, \
        masks_tensors, labels = [t.to(device) for t in data]

        # 將參數梯度歸零
        optimizer.zero_grad()
        
        # forward pass
        outputs = model(input_ids=tokens_tensors, 
                        token_type_ids=segments_tensors, 
                        attention_mask=masks_tensors, 
                        labels=labels)

        loss = outputs[0]
        # backward
        loss.backward()
        optimizer.step()


        # 紀錄當前 batch loss
        running_loss += loss.item()
        
    # 計算分類準確率
    _, acc = get_predictions(model, trainloader, compute_acc=True)

    print('[epoch %d] loss: %.3f, acc: %.3f' %
          (epoch + 1, running_loss, acc))
    

[epoch 1] loss: 53.253, acc: 0.982
[epoch 2] loss: 39.645, acc: 0.982
[epoch 3] loss: 27.857, acc: 0.984
[epoch 4] loss: 19.209, acc: 0.992
[epoch 5] loss: 15.208, acc: 0.977
[epoch 6] loss: 9.326, acc: 0.975
CPU times: user 8min 47s, sys: 3min 36s, total: 12min 23s
Wall time: 12min 26s


In [40]:
date="20210821"
path="/content/drive/MyDrive/BertModel/bert20210821.pth"

torch.save(model.state_dict(),path)
#PATH="/content/drive/MyDrive/BertModel/bert20210821.pth"
#checkpoint = torch.load(PATH)
#model.load_state_dict(checkpoint)